In [4]:
from neuron_morphology.feature_extractor.data import Data

from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor
from neuron_morphology.features import dimension, intrinsic
from tests.objects import test_morphology_large




### Create Marked Functions from imports

In [5]:
from neuron_morphology.feature_extractor.data import Data
from neuron_morphology.feature_extractor.marked_feature import marked
from neuron_morphology.feature_extractor.mark import (
    RequiresLayerAnnotations, Intrinsic, Geometric, AllNeuriteTypes,
    RequiresSoma)

# Mark the preexisting calculate_dimension_features_for_all_types function
from neuron_morphology.features import dimension
@marked(AllNeuriteTypes)
@marked(Geometric)
def dimension_features(data: Data):
    return dimension.calculate_dimension_features_for_all_types(data.morphology)

# Mark the preexisting calculate_intrinsic_features_for_all_types function
from neuron_morphology.features import intrinsic
@marked(AllNeuriteTypes)
@marked(Intrinsic)
def intrinsic_features(data: Data):
    return intrinsic.calculate_intrinsic_features_for_all_types(data.morphology)

# Create a new feature function
@marked(RequiresSoma)
@marked(Geometric)
def example_soma_features(data: Data):
    soma_node = data.morphology.get_root()
    features = {'soma': {'x': soma_node['x'],
                         'y': soma_node['y'],
                         'z': soma_node['z']}}
    return features

### Build a Feature Extractor from those marked functions

In [9]:
from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor

# Add features One by one
fe = FeatureExtractor()
fe.register_features([dimension_features])
fe.register_features([intrinsic_features])
fe.register_features([example_soma_features])

# or All at once: 
#fe = FeatureExtractor((dimension_features, intrinsic_features, example_soma_features))

### Import some data

In [10]:
from tests.objects import test_morphology_large
data = Data(test_morphology_large()) 


### Run the feature extractor on the data

In [11]:
feature_extraction_run = fe.extract(data)
results = feature_extraction_run.results

### Check the results

In [12]:
results

{'intrinsic_features': {'all_neurites': {'num_branches': 3,
   'max_branch_order': 0,
   'mean_fragmentation': nan,
   'num_tips': 3,
   'num_nodes': 17},
  'axon': {'num_branches': 1,
   'max_branch_order': 0,
   'mean_fragmentation': nan,
   'num_tips': 1,
   'num_nodes': 6},
  'basal_dendrite': {'num_branches': 1,
   'max_branch_order': 0,
   'mean_fragmentation': nan,
   'num_tips': 1,
   'num_nodes': 4},
  'apical_dendrite': {'num_branches': 1,
   'max_branch_order': 0,
   'mean_fragmentation': nan,
   'num_tips': 1,
   'num_nodes': 6}},
 'example_soma_features': {'soma': {'x': 800, 'y': 610, 'z': 30}},
 'dimension_features': {'all_neurites': {'width': 650,
   'height': 450,
   'depth': 70,
   'min_x': 400,
   'min_y': 300,
   'min_z': 10,
   'max_x': 1050,
   'max_y': 750,
   'max_z': 80},
  'axon': {'width': 150,
   'height': 150,
   'depth': 50,
   'min_x': 900,
   'min_y': 600,
   'min_z': 30,
   'max_x': 1050,
   'max_y': 750,
   'max_z': 80},
  'basal_dendrite': {'width': 90